In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import iSwapGate, XXPlusYYGate, CXGate
from canopus.utils import qiskit_to_tket, tket_to_qiskit
import canopus
from qiskit.converters import circuit_to_dag, dag_to_circuit

In [2]:
qc = QuantumCircuit(2)
qc.iswap(0, 1)
qc.h(0)
qc.h(1)
qc.cx(1, 0)
qc.append(iSwapGate().power(-1), [0, 1])
qc.draw()

┌────────┐┌───┐┌───┐┌───────────────┐
q_0: ┤0       ├┤ H ├┤ X ├┤0              ├
     │  Iswap │├───┤└─┬─┘│  (XX+YY)(π,0) │
q_1: ┤1       ├┤ H ├──■──┤1              ├
     └────────┘└───┘     └───────────────┘

In [3]:
dag = circuit_to_dag(qc)

In [4]:
dag.remove_all_ops_named('h')

In [5]:
dag_to_circuit(dag).draw()

┌────────┐┌───┐┌───────────────┐
q_0: ┤0       ├┤ X ├┤0              ├
     │  Iswap │└─┬─┘│  (XX+YY)(π,0) │
q_1: ┤1       ├──■──┤1              ├
     └────────┘     └───────────────┘

In [31]:
set(qc.data[3].qubits)

{Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)}

In [22]:
c = canopus.rebase_to_tk2(qiskit_to_tket(qc))
c = canopus.rebase_to_zzphase(c)
tket_to_qiskit(c).draw()

┌─────────────────┐           ┌────────────────┐           »
q_0: ┤ U3(0,-π/2,3π/2) ├─■─────────┤ U3(π/2,-π/2,π) ├──■────────»
     └─┬──────────────┬┘ │ZZ(π/2) ┌┴────────────────┴┐ │ZZ(π/2) »
q_1: ──┤ U3(π/2,0,4π) ├──■────────┤ U3(5π/2,-π/2,4π) ├─■────────»
       └──────────────┘           └──────────────────┘          »
«     ┌────────────────┐
«q_0: ┤ U3(5π/2,0,π/2) ├
«     ├────────────────┤
«q_1: ┤ U3(3π,-π/2,4π) ├
«     └────────────────┘

In [14]:
from pytket.utils.stats import gate_counts

gate_counts(qiskit_to_tket(qc))

Counter({OpType.CustomGate: 2, OpType.H: 2})

In [16]:
c

[TK1(0, 0, 1) q[0]; TK1(0.5, 0.5, 3.5) q[1]; ZZPhase(0.5) q[0], q[1]; TK1(0, 0.5, 0.5) q[0]; TK1(0, 2.5, 3.5) q[1]; ZZPhase(0.5) q[0], q[1]; TK1(0.5, 2.5, 0) q[0]; TK1(0, 3, 3.5) q[1]; ]

In [19]:
tket_to_qiskit(c).data[2]

CircuitInstruction(operation=Instruction(name='rzz', num_qubits=2, num_clbits=0, params=[1.5707963267948966]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())

In [21]:
qc.data[0]

CircuitInstruction(operation=Instruction(name='iswap', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())

In [22]:
qc.data[-1]

CircuitInstruction(operation=Instruction(name='xx_plus_yy', num_qubits=2, num_clbits=0, params=[3.141592653589793, 0]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())

In [19]:
CXGate().power(-1)

Instruction(name='unitary', num_qubits=2, num_clbits=0, params=[array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]])])

In [ ]:
qc.qubits[0].

AttributeError: 'Qubit' object has no attribute 'index'

In [28]:
import pytket

In [29]:
c = pytket.Circuit(2)

In [31]:
c.qubits[0]

q[0]

In [10]:
from qiskit.circuit.library import QFT, CPhaseGate
from canopus import rebase_to_tk2

qc = QFT(4, do_swaps=False).decompose()
circ = qiskit_to_tket(qc)
circ = rebase_to_tk2(circ)
qc = tket_to_qiskit(circ)

In [11]:
dag = circuit_to_dag(qc)
dag.remove_all_ops_named('u')
dag.op_nodes()

[DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.7853981633974483, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(4, 'q'), 2), Qubit(QuantumRegister(4, 'q'), 3)), cargs=()),
 DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.3926990816987245, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(4, 'q'), 1), Qubit(QuantumRegister(4, 'q'), 3)), cargs=()),
 DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.19634954084936243, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(4, 'q'), 0), Qubit(QuantumRegister(4, 'q'), 3)), cargs=()),
 DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.7853981633974483, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(4, 'q'), 1), Qubit(QuantumRegister(4, 'q'), 2)), cargs=()),
 DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.3926990816987245, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(4, 'q'), 0), Qubit(QuantumRegister(4, 'q'), 2)), cargs=()),
 DAGOpNode(op=Instructio